In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
import os
from IPython.display import display_html
import marko

from taogpt.orchestrator import *
from taogpt.utils import *
from taogpt.tao_model import LangChainLLM
from taogpt.prompts import PromptSet
import taogpt.utils as utils

utils.enable_debugging(0)

In [5]:
GPT_MODEL = 'gpt-4-32k'
TEMPERATURE = 0.7

In [ ]:
with open(os.path.join(os.environ['HOME'], '.ssh', 'openai-zillow.key'), 'r') as f:
    key = ''.join(f.readlines()).strip()
os.environ["OPENAI_API_KEY"] = key
os.environ["OPENAI_API_BASE"] = "https://zgai-llm-api.int.stage-k8s.zg-aip.net/openai/v1"
llm = ChatOpenAI(model_name=GPT_MODEL, temperature=TEMPERATURE)

conversation = ConversationChain(llm=llm)
conversation.predict(input="What's your model version?")

In [ ]:
prompts = PromptSet.load_defaults()
logger = MarkdownLogger('logs/taogpt_log.md')
executor = Orchestrator(llm=LangChainLLM(llm, logger=logger), prompts=prompts, markdown_logger=logger,
                        max_tokens=200000)

experiment_name = 'example'
executor.start("""
""", try_intuition=True)

In [ ]:
logger = MarkdownLogger(f'examples/{experiment_name}.final.md')
logger.log_conversation(executor.show_conversation_thread(with_extras=True))
logger.log(f"**total tokens**: {executor.llm.total_tokens}")

In [ ]:
with open(logger._log_path, 'r') as f:
    html_output = marko.convert(f.read())
    display_html(html_output, raw=True)